# Обобщение текста

In [1]:
# !pip install torch transformers

Библиотека transformers предоставляет большое количество моделей для работы с разными типами данных для огромного спектра задач: 
1. Обобщение текста
2. Сегментация
3. Классификация
4. Перевод текста
5. Обобщение
6. Генерация

Сегодня нас интересуют модели для работы с текстами. 

In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

# Отключение предупреждений
__import__("warnings").filterwarnings("ignore")

# Проверка наличия GPU с помощью PyTorch
if torch.cuda.is_available():
    device = 'cuda' 
    print("GPU доступен. Использую CUDA для обчения!!! :)")
else:
    device = "cpu"
    print("В вашей системе нет GPU :( Обучаюсь на CPU.")

GPU доступен. Использую CUDA для обчения!!! :)


In [24]:
model_name = "IlyaGusev/rugpt3medium_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [25]:
config = AutoConfig.from_pretrained(model_name)
max_length = config.max_position_embeddings
print(f"Максимальная длина последовательности для этой модели: {max_length} токенов")

Максимальная длина последовательности для этой модели: 2048 токенов


Эта модель работает следующим образом: получает на вход какой-то текст и выводит общую идею этой статьи.

In [32]:
article_text = "Актуальность проблемы. Электронная информация играет все большую роль во всех сферах жизни современного общества. В последние годы объем научно-технической текстовой информации в электронном виде возрос настолько, что возникает угроза обесценивания этой информации в связи с трудностями поиска необходимых сведений среди множества доступных текстов. Развитие информационных ресурсов Интернет многократно усугубило проблему информационной перегрузки. В этой ситуации особенно актуальными становятся методы автоматизации реферирования текстовой информации, то есть методы получения сжатого представления текстовых документов–рефератов (аннотаций). Постановка проблемы автоматического реферирования текста и, соответственно, попытки ее решения с использованием различных подходов предпринимались многими исследователями."
print(article_text)

Актуальность проблемы. Электронная информация играет все большую роль во всех сферах жизни современного общества. В последние годы объем научно-технической текстовой информации в электронном виде возрос настолько, что возникает угроза обесценивания этой информации в связи с трудностями поиска необходимых сведений среди множества доступных текстов. Развитие информационных ресурсов Интернет многократно усугубило проблему информационной перегрузки. В этой ситуации особенно актуальными становятся методы автоматизации реферирования текстовой информации, то есть методы получения сжатого представления текстовых документов–рефератов (аннотаций). Постановка проблемы автоматического реферирования текста и, соответственно, попытки ее решения с использованием различных подходов предпринимались многими исследователями.


Посмотрим, в чем, по мнению нашей модели, суть этого текста.

In [33]:
def get_summary(text, min_tokens=20, max_new_tokens=80):
    text_tokens = tokenizer(
        text,
        max_length=1024, # Обрезка входного текста до 1024 токенов
        add_special_tokens=False,
        padding=False,
        truncation=True
    )["input_ids"]

    input_ids = text_tokens + [tokenizer.sep_token_id]
    input_ids = torch.LongTensor([input_ids]).to(device)
    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4,
        max_new_tokens=max_new_tokens,  # Максимальное количество новых токенов
        min_length=min_tokens   # Минимальная длина генерируемого текста
    )

    summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
    summary = summary.split(tokenizer.sep_token)[1]
    summary = summary.split(tokenizer.eos_token)[0]
    return summary

In [34]:
summary = get_summary(article_text)
print(f"Обобщение:\n{summary}")

Обобщение:
Электронная информация становится все более значимой в жизни современного общества, и это не может не вызывать озабоченности.


In [35]:
article_text = "Попрыгунья Стрекоза \
Лето красное пропела; \
Оглянуться не успела, \
Как зима катит в глаза. \
Помертвело чисто поле; \
Нет уж дней тех светлых боле, \
Как под каждым ей листком \
Был готов и стол, и дом. \
Все прошло: с зимой холодной \
Нужда, голод настает; \
Стрекоза уж не поет: \
И кому же в ум придет \
На желудок петь голодный! \
Злой тоской удручена, \
К Муравью ползет она: \
«Не оставь меня, кум милый! \
Дай ты мне собраться с силой \
И до вешних только дней \
Прокорми и обогрей!» — \
«Кумушка, мне странно это: \
Да работала ль ты в лето?» — \
Говорит ей Муравей. \
«До того ль, голубчик, было? \
В мягких муравах у нас \
Песни, резвость всякий час, \
Так, что голову вскружило». — \
«А, так ты...» — «Я без души \
Лето целое все пела». — \
«Ты все пела? Это дело: \
Так поди же, попляши!»"


article_text

'Попрыгунья Стрекоза Лето красное пропела; Оглянуться не успела, Как зима катит в глаза. Помертвело чисто поле; Нет уж дней тех светлых боле, Как под каждым ей листком Был готов и стол, и дом. Все прошло: с зимой холодной Нужда, голод настает; Стрекоза уж не поет: И кому же в ум придет На желудок петь голодный! Злой тоской удручена, К Муравью ползет она: «Не оставь меня, кум милый! Дай ты мне собраться с силой И до вешних только дней Прокорми и обогрей!» — «Кумушка, мне странно это: Да работала ль ты в лето?» — Говорит ей Муравей. «До того ль, голубчик, было? В мягких муравах у нас Песни, резвость всякий час, Так, что голову вскружило». — «А, так ты...» — «Я без души Лето целое все пела». — «Ты все пела? Это дело: Так поди же, попляши!»'

In [38]:
summary = get_summary(article_text)
print(summary)

Муравей и Стрекоза, Муравей и Муравей, Стрекоза и Муравей — что общего между этими двумя персонажами из сказки братьев Гримм «Стрекоза и муравей» и чем они друг от друга отличаются?


# Задание 3

Попробуй применить нейронную сеть для других текстов. Посмотри, для каких текстов получилось хорошо, для каких не очень.

In [47]:
texts = [
"""Современные технологии шагнули далеко вперед. Теперь написать конспект можно с помощью искусственного интеллекта. Это значительно упрощает процесс изучения материала. Нейросети анализируют текст, выделяют ключевые моменты и идеи. Таким образом, они формируют краткое содержание текста (выжимку контента онлайн).
Нейросеть для конспектирования ChatInfo экономит ваше время. Вам не нужно перечитывать весь материал. Просто искусственный интеллект сделает это за вас. Поэтому, вы можете сосредоточиться на других задачах. Кроме того, такой подход минимизирует риск пропуска важной информации.
Изложение текста нейросетью обеспечивает высокую точность. Нейросети обучены распознавать смысловые связи. В частности, они умеют определять главную мысль. Кроме этого, они выявляют второстепенные детали. Это позволяет получить полное представление о тексте без лишних подробностей.""",
"""Сервисы для создания выжимки текста онлайн становятся все популярнее. Они предоставляют возможность быстро получить суть любого материала. Это особенно ценно в условиях информационного перегруза. Краткое содержание текста дает возможность оценить его значимость без глубокого погружения.
Выжимка текста онлайн нейросетью помогает сфокусироваться на главном. То есть, она избавляет от необходимости вникать во все детали. Это экономит время и усилия. Кроме того, сервис конспектирования ChatInfo подойдет для подготовки презентаций или докладов.""",
"""Генераторы названий брендов нейросетью — это отличный инструмент для создания уникального и запоминающегося нейминга. Они основаны на алгоритмах машинного обучения. Поэтому их названия могут быть полностью новыми и необычными, но при этом легко читаемыми и простыми для запоминания.
Ключевым преимуществом нейросети генератора брендов является ее способность генерировать огромное количество различных вариантов в считанные секунды. Это не только экономит время, но также позволяет избежать повторения уже существующих именований.""",
"""Разведочный анализ данных, или EDA, – это как археологические раскопки в мире информации. Это первый шаг, когда мы берем на себя роль исследователя данных и начинаем расследовать, как устроены наши данные, как они взаимосвязаны и что они нам могут рассказать. EDA – это не просто скучная предварительная обработка, это настоящее приключение, в ходе которого мы обнаруживаем неожиданные моменты, паттерны и закономерности, которые часто прячутся на первый взгляд.
Помимо этого, EDA – это наша страховка от неприятных сюрпризов. Мы находим аномалии данных, которые могут исказить наши выводы, и можем предпринять меры для их коррекции. Также EDA помогает нам определить, какие данные нам не хватает, чтобы получить полное представление о ситуации, и планомерно заполнить эти пробелы""",

]
for idx, text in enumerate(texts, 1):
    print(f'Текст № {idx}:\n{text}')
    summary = get_summary(text)
    print(f"\nОбобщение:\n{summary}\n")

Текст № 1:
Современные технологии шагнули далеко вперед. Теперь написать конспект можно с помощью искусственного интеллекта. Это значительно упрощает процесс изучения материала. Нейросети анализируют текст, выделяют ключевые моменты и идеи. Таким образом, они формируют краткое содержание текста (выжимку контента онлайн).
Нейросеть для конспектирования ChatInfo экономит ваше время. Вам не нужно перечитывать весь материал. Просто искусственный интеллект сделает это за вас. Поэтому, вы можете сосредоточиться на других задачах. Кроме того, такой подход минимизирует риск пропуска важной информации.
Изложение текста нейросетью обеспечивает высокую точность. Нейросети обучены распознавать смысловые связи. В частности, они умеют определять главную мысль. Кроме этого, они выявляют второстепенные детали. Это позволяет получить полное представление о тексте без лишних подробностей.

Обобщение:
Современные технологии шагают в ногу со временем. Теперь конспектировать можно с помощью нейросетей. Это

1. Суть передана верно.
2. Суть передата приблизительно, добавлена отсебятина.
3. Выжимка из текста близка к галюцинации. Должно быть что-то типа:
   Нейросеть на основе алгоритмов ML генерирует запоминающиеся названия брендов, не повторяясь и экономя нам время.
4. Начало вполне нормальное, окончание предложения - галюцинация.

По пятибальной шкале: удовлетворительно, т.к. используется довольно старая модель GPT2.